In [2]:
import pandas as pd
import numpy as np
import sys

In [ ]:
# 规范化数据部分
pan_train = pd.read_csv("./DataSet/train.csv", header=None)
pan_test = pd.read_csv("./DataSet/test.csv", header=None)
pan_train.drop([pan_train.columns[247]], axis=1,inplace=True)
pan_test.drop([pan_test.columns[246]], axis=1,inplace=True)
pan_train_label = pan_train.iloc[:, 246]
pan_train_data = pan_train.iloc[:, 0:246]
data_all = pd.concat([pan_train_data, pan_test], axis=0)

pan_train_label.to_csv('./DataSet/train_label.csv',header=False, index=False)
data_all.to_csv('./DataSet/data_all.csv', header=False, index=False)
pan_train_data.to_csv('./DataSet/train_data.csv',header=False, index=False)
pan_test.to_csv('./DataSet/test_data.csv',header=False, index=False)

In [ ]:
# 清洗数据
data_all = pd.read_csv('./DataSet/data_all.csv', header=None)
train_data = pd.read_csv('./DataSet/train_data.csv',header=None)
test_data = pd.read_csv('./DataSet/test_data.csv',header=None)
train_label = pd.read_csv('./DataSet/train_label.csv',header=None)
data_all = data_all.dropna(axis=1, how='all')
data_all = data_all.fillna(data_all.mean())
data_all.to_csv('./DataSet/data_all_1FillWithMean.csv', header=False, index=False)
for i in data_all.columns:
    d=len(data_all)-data_all[i].count()
    r=(d/len(data_all))*100
    rate='%.2f%%' % r
    if d != 0:
        print('字段名为：',str(i).ljust(10),'缺失值数量:',str(d).ljust(4),'缺失数量占比：',rate)
        if r < 1:
            word = data_all.iloc[:, i].mode().loc[0]
            data_all.iloc[:, i].fillna(word, inplace=True)
        if r > 1:
            data_all.iloc[:, i].fillna('missing', inplace=True)
data_all.to_csv('./DataSet/data_all_2Full.csv', header=False, index=False)

In [ ]:
#时间转换为数值
data_all = pd.read_csv('./DataSet/data_all_2Full.csv',header=None)
time_t = pd.to_datetime(data_all.iloc[:,198], format='%Y-%m-%d-%H.%M.%S.%f')
data_all.drop(198, 1, inplace=True)
min_t = min(time_t)
list_day = [(i-min_t).days for i in time_t]
days = pd.DataFrame(list_day)
data_all = pd.concat([data_all, days], axis=1)
data_all.to_csv('./DataSet/data_all_3Full_with_day.csv', header=False, index=False)

In [ ]:
# 以时间分块
data_all = pd.read_csv('./DataSet/data_all_3Full_with_day.csv',header=None)
temp = pd.get_dummies(data_all.iloc[:, 237])
data_all.drop(237, 1, inplace=True)
data_all = pd.concat([data_all, temp], axis=1)
data_all.to_csv('./DataSet/data_all_4Full_with_day_dummies.csv', header=False, index=False)

In [3]:
# 全部转成数字
data_all = pd.read_csv('./DataSet/data_all_4Full_with_day_dummies.csv',header=None)
for i in range(len(data_all.columns)):
    ty = data_all.iloc[:,i]
    type_  = (str(ty.dtype))
    if type_ != 'float64' and type_ != 'int64':
        temp = pd.get_dummies(ty)
        data_all = pd.concat([data_all, temp], axis=1)
for i in range(300):
    ty = data_all.loc[:,i]
    type_  = (str(ty.dtype))
    if type_ != 'float64' and type_ != 'int64':
        data_all.drop(i, 1, inplace=True)
data_all.to_csv('./DataSet/data_all_5ALL_Digtal.csv', header=False, index=False)

In [4]:
# 还原成train data和test data
data_all = pd.read_csv('./DataSet/data_all_5ALL_Digtal.csv',header=None)
train_label = pd.read_csv('./DataSet/train_label.csv',header=None)
train_count = len(train_label)
train_data = data_all.iloc[:train_count, :]
test_data = data_all.iloc[train_count:, :]
train_data.to_csv('./DataSet/Last_Train.csv',header=False, index=False)
test_data.to_csv('./DataSet/Last_Test.csv',header=False, index=False)
train_label.to_csv('./DataSet/Last_Train_Label.csv',header=False, index=False)